In [1]:
import numpy as np
import pandas as pd
import os.path
from sklearn.metrics import roc_auc_score
import tqdm

# Santander Customer Transaction Prediction (KAGGLE)

Leer de que trata la competencia

https://www.kaggle.com/c/santander-customer-transaction-prediction/overview

# Bajada de Dataset
La siguiente celda baja el dataset y lo guarda en la carpeta /data.

Luego de ejecutarla debería tener los archivos train.csv y test.csv en esta carpeta

# Carpeta on dataset:
https://drive.google.com/drive/folders/19scmxZ64fe7mIneDwG21eF_QLn-8-S8j?usp=sharing

In [2]:
# Si quiero habilitar mi google drive descomentar
# from google.colab import drive
# drive.mount('/content/gdrive')

# Cargar datos de train

In [3]:
df_train = pd.read_csv('data/lab-2/train.csv')

In [4]:
# TODO: Importar datos de test y asignarlos a la variable df_test desde la carpeta data/
df_test  = pd.read_csv('data/lab-2/test.csv')
# Miramos las primeras 5 observaciones del dataset
df_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [5]:
df_train.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)

#### Cuantas features (covariables, variable explicativa, variable independiente, variables exógenas) hay (dimensión de las observaciones, regresoras)? 
El dataset contiene 200 features (var_0 a var_199).  
#### Que columna contiene la salida (variable dependiente, variable endógena)? 
La salida se encuentra en la columna `target`
#### Cuantas clases hay? 
Hay 2 clases, etiquetadas como 0 y 1
#### Cuantas observaciones tiene train?
El dataset de entrenamiento contiene 200,000 observaciones.

In [6]:
df_train.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [7]:
# TODO: Inspeccionar test
# Puede ejecutar los métodos que crea necesarios. Recomendamos describe, columns, shape, head

In [8]:
df_test.describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,10.658737,-1.624244,10.707452,6.788214,11.076399,-5.050558,5.415164,16.529143,0.277135,7.569407,...,3.189766,7.458269,1.925944,3.322016,17.996967,-0.133657,2.290899,8.912428,15.869184,-3.246342
std,3.036716,4.040509,2.633888,2.052724,1.616456,7.869293,0.864686,3.424482,3.333375,1.231865,...,4.551239,3.025189,1.479966,3.995599,3.140652,1.429678,5.446346,0.920904,3.008717,10.398589
min,0.188700,-15.043400,2.355200,-0.022400,5.484400,-27.767000,2.216400,5.713700,-9.956000,4.243300,...,-14.093300,-2.407000,-3.340900,-11.413100,9.382800,-4.911900,-13.944200,6.169600,6.584000,-39.457800
25%,8.442975,-4.700125,8.735600,5.230500,9.891075,-11.201400,4.772600,13.933900,-2.303900,6.623800,...,-0.095000,5.166500,0.882975,0.587600,15.634775,-1.160700,-1.948600,8.260075,13.847275,-11.124000
50%,10.513800,-1.590500,10.560700,6.822350,11.099750,-4.834100,5.391600,16.422700,0.372000,7.632000,...,3.162400,7.379000,1.892600,3.428500,17.977600,-0.162000,2.403600,8.892800,15.943400,-2.725950
75%,12.739600,1.343400,12.495025,8.327600,12.253400,0.942575,6.005800,19.094550,2.930025,8.584825,...,6.336475,9.531100,2.956000,6.174200,20.391725,0.837900,6.519800,9.595900,18.045200,4.935400
max,22.323400,9.385100,18.714100,13.142000,16.037100,17.253700,8.302500,28.292800,9.665500,11.003600,...,20.359000,16.716500,8.005000,17.632600,27.947800,4.545400,15.920700,12.275800,26.538400,27.907400


In [9]:
df_test.shape

(200000, 201)

#### Que columna esta en los datos de train pero no en los de test?
La columna `target`
#### Cuantas observaciones de test hay?
El dataset de test tiene la misma cantidad de observaciones que train, 200,000

In [10]:
df_train.target.value_counts()/len(df_train)

0    0.89951
1    0.10049
Name: target, dtype: float64

# Dividir data en Train y Val

In [11]:
N_train = 160_000
X_train = df_train.drop(columns=['ID_code', 'target']).values[:N_train]
X_val = df_train.drop(columns=['ID_code', 'target']).values[N_train:]
y_train = df_train['target'].values[:N_train]
y_val = df_train['target'].values[N_train:]

# Regresión logística

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
model = LogisticRegression(solver='lbfgs', max_iter = 100)

In [14]:
%time model.fit(X_train, y_train)

Wall time: 7.51 s


C:\Users\jvidela\miniconda3\envs\diplo\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

#### A que se debe el warning no termina de convergence?
El modelo no termina de converger debido a que se alcanzó el número máximo de iteraciones (100)

In [15]:
acc = model.score(X_train, y_train)
auc = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
print(acc)
print(auc)

0.91204375
0.85029461143855


In [16]:
print(roc_auc_score(y_val, model.predict_proba(X_val)[:,1]))
print(model.score(X_val, y_val))

0.8525540158701921
0.912425


cuantos parámetros aprendio el modelo?

In [17]:
print(model.coef_.shape)
print(model.intercept_.shape)

(1, 200)
(1,)


El modelo aprendió 201 parámetros: 1 por cada feature (200) + Intercept

In [18]:
model = LogisticRegression(solver='lbfgs', max_iter = 10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [19]:
print(model.score(X_val, y_val))
print(roc_auc_score(y_val, model.predict_proba(X_val)[:,1]))

0.913925
0.8627138133096955


# Normalizar los datos

In [20]:
means = X_train.mean(axis=0)
stds = X_train.std(axis=0)

In [21]:
X_train_normalized = (X_train - means)/stds
X_val_normalized = (X_val - means)/stds

In [22]:
X_train_normalized

array([[-0.57581888, -1.27213281,  0.45287052, ..., -0.37754551,
        -1.02664168,  0.21435054],
       [ 0.2712627 , -0.62084311,  1.19211864, ..., -0.13305653,
         0.82532811,  0.50563783],
       [-0.67983317, -0.27493645,  0.51820418, ..., -0.69861357,
        -0.38163924,  0.35676851],
       ...,
       [-0.36992848,  0.29836631, -1.02369299, ...,  1.02268001,
         1.20524149,  0.97469155],
       [-0.85496666, -0.23515324,  0.40515876, ...,  0.58630947,
        -1.17760394,  1.52298016],
       [ 0.34159241, -0.12772376, -0.69278026, ..., -1.1042874 ,
         0.05643364,  0.39886503]])

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
scaler = StandardScaler()
scaler.fit_transform(X_train)

array([[-0.57581888, -1.27213281,  0.45287052, ..., -0.37754551,
        -1.02664168,  0.21435054],
       [ 0.2712627 , -0.62084311,  1.19211864, ..., -0.13305653,
         0.82532811,  0.50563783],
       [-0.67983317, -0.27493645,  0.51820418, ..., -0.69861357,
        -0.38163924,  0.35676851],
       ...,
       [-0.36992848,  0.29836631, -1.02369299, ...,  1.02268001,
         1.20524149,  0.97469155],
       [-0.85496666, -0.23515324,  0.40515876, ...,  0.58630947,
        -1.17760394,  1.52298016],
       [ 0.34159241, -0.12772376, -0.69278026, ..., -1.1042874 ,
         0.05643364,  0.39886503]])

In [25]:
model_2 = LogisticRegression(solver='lbfgs')
%time model_2.fit(X_train_normalized, y_train)

Wall time: 2.88 s


LogisticRegression()

In [26]:
print(model_2.score(X_train_normalized, y_train))
print(roc_auc_score(y_train, model_2.predict_proba(X_train_normalized)[:,1]))

0.91440625
0.8606427353156717


In [27]:
print(model_2.score(X_val_normalized, y_val))
print(roc_auc_score(y_val, model_2.predict_proba(X_val_normalized)[:,1]))

0.914525
0.8629075884886785


In [28]:
# Ordenamos el valor absoluto de los parámetros de mayor a menor
# Coeficientes ordenados en función de su de mayor importancia
np.argsort(np.abs(model_2.coef_))[0][::-1]

array([ 81, 139,   6,  12,  53, 110,  76,  99, 146, 174,  26,  21,  22,
        80, 190, 166, 165, 198,  13,  34, 169,  44,   2, 148,   0,  40,
       133, 179, 170,  78,  94,   1, 184, 109,  33, 115,  92,  67, 149,
       108, 191, 122, 173, 154,  18,  86, 192, 118, 107, 121, 147,  95,
         9,  75,  35, 164, 177, 197, 172,  36, 127,  89, 123, 155,  91,
       188,  56,  87,  71,  48,   5,  93, 162, 106, 157, 130, 141, 145,
        24, 151,  32, 167, 163, 150, 186, 119,  49,  31, 180,  23, 111,
       195,  90, 131, 125, 137, 114, 199,  43, 116, 135,  52,  58, 128,
        70, 104, 175, 112, 132, 105,  11, 196,  85,  82, 194,  51,  28,
       142,  83,  66, 134, 144, 138,  74,  45, 156,  77,  55,  97, 140,
        20,  54, 193,  57,  88, 178,   8, 102, 113,  62,  15, 143, 159,
       187, 181, 171,  63,  72,  64,  50,  59, 120, 168, 182, 101,  25,
        68,   3,  65, 152,   4,  84,  37,  42, 176,  61, 129,  19, 189,
        69,  47,  60,  16,  27,  29,  79,  73, 158,  96, 160,  4

In [29]:
# Parámetros menores a cero
negative_indexes = np.where(model.coef_<0)[1]
negative_indexes

array([  7,   9,  10,  12,  13,  14,  17,  20,  21,  23,  27,  28,  30,
        31,  33,  34,  36,  38,  39,  41,  42,  43,  44,  45,  50,  54,
        56,  57,  58,  59,  63,  64,  72,  73,  75,  76,  77,  80,  81,
        83,  85,  86,  87,  88,  92,  93,  98, 101, 102, 103, 104, 107,
       108, 109, 113, 114, 115, 116, 120, 121, 122, 123, 127, 129, 131,
       132, 136, 139, 141, 142, 143, 146, 148, 149, 150, 152, 153, 154,
       156, 158, 160, 165, 166, 169, 172, 174, 177, 178, 182, 183, 185,
       186, 188, 192, 193, 194, 197, 198], dtype=int64)

In [30]:
# Parámetros mayores a cero
positive_indexes = np.where(model.coef_>0)[1]
positive_indexes

array([  0,   1,   2,   3,   4,   5,   6,   8,  11,  15,  16,  18,  19,
        22,  24,  25,  26,  29,  32,  35,  37,  40,  46,  47,  48,  49,
        51,  52,  53,  55,  60,  61,  62,  65,  66,  67,  68,  69,  70,
        71,  74,  78,  79,  82,  84,  89,  90,  91,  94,  95,  96,  97,
        99, 100, 105, 106, 110, 111, 112, 117, 118, 119, 124, 125, 126,
       128, 130, 133, 134, 135, 137, 138, 140, 144, 145, 147, 151, 155,
       157, 159, 161, 162, 163, 164, 167, 168, 170, 171, 173, 175, 176,
       179, 180, 181, 184, 187, 189, 190, 191, 195, 196, 199], dtype=int64)

In [31]:
len(negative_indexes)/len(positive_indexes)

0.9607843137254902

Notar que hay algunos parámetros mayores a cero y otros menores a cero. Como contribuyen en función de su signo?

In [32]:
# Ordenados de menor a mayor
np.argsort(model_2.coef_)[0]

array([ 81, 139,  12,  76, 146, 174,  21,  80, 166, 165, 198,  13,  34,
       169,  44, 148, 109,  33, 115,  92, 149, 108, 122, 154,  86, 192,
       107, 121,   9,  75, 177, 197, 172,  36, 127, 123, 188,  56,  87,
        93, 141, 150, 186,  31,  23, 131, 114,  43, 116,  58, 104, 132,
        85, 194,  28, 142,  83,  45, 156,  77,  20,  54, 193,  57,  88,
       178, 102, 113, 143,  63,  72,  64,  50,  59, 120, 182, 101,  68,
       152,  42, 129,  27,  73, 158, 160,  98,  14, 153,  39, 136, 183,
        41, 103,  10,   7, 185,  38,  17,  30, 161, 100, 117, 126, 124,
        46,  96,  79,  29,  16,  60,  47,  69, 189,  19,  61, 176,  37,
        84,   4,  65,   3,  25, 168, 171, 181, 187, 159,  15,  62,   8,
       140,  97,  55,  74, 138, 144, 134,  66,  51,  82, 196,  11, 105,
       112, 175,  70, 128,  52, 135, 199, 137, 125,  90, 195, 111, 180,
        49, 119, 163, 167,  32, 151,  24, 145, 130, 157, 106, 162,   5,
        48,  71,  91, 155,  89, 164,  35,  95, 147, 118,  18, 17

In [33]:
# Ordenados de mayor a menor
np.argsort(model_2.coef_)[0][::-1]

array([  6,  53, 110,  99,  26,  22, 190,   2,   0,  40, 133, 179, 170,
        78,  94,   1, 184,  67, 191, 173,  18, 118, 147,  95,  35, 164,
        89, 155,  91,  71,  48,   5, 162, 106, 157, 130, 145,  24, 151,
        32, 167, 163, 119,  49, 180, 111, 195,  90, 125, 137, 199, 135,
        52, 128,  70, 175, 112, 105,  11, 196,  82,  51,  66, 134, 144,
       138,  74,  55,  97, 140,   8,  62,  15, 159, 187, 181, 171, 168,
        25,   3,  65,   4,  84,  37, 176,  61,  19, 189,  69,  47,  60,
        16,  29,  79,  96,  46, 124, 126, 117, 100, 161,  30,  17,  38,
       185,   7,  10, 103,  41, 183, 136,  39, 153,  14,  98, 160, 158,
        73,  27, 129,  42, 152,  68, 101, 182, 120,  59,  50,  64,  72,
        63, 143, 113, 102, 178,  88,  57, 193,  54,  20,  77, 156,  45,
        83, 142,  28, 194,  85, 132, 104,  58, 116,  43, 114, 131,  23,
        31, 186, 150, 141,  93,  87,  56, 188, 123, 127,  36, 172, 197,
       177,  75,   9, 121, 107, 192,  86, 154, 122, 108, 149,  9

In [34]:
# Calculamos correlación de cada variable con la salida
correlations = []
for i in range(200):
    correlations.append(np.corrcoef(y_train, X_train_normalized[:,i])[1,0])

In [36]:
np.argsort(np.abs(correlations))[::-1]

array([ 81, 139,  12, 110,   6,  26,  53, 146, 174,  76,  99,  80, 166,
        22, 165,  21, 190, 148,   2,  13,  34, 133,   0, 198, 169, 109,
       179,  44,  40,   1, 115, 149, 170, 184,  78,  94,  92, 108,  67,
       191,  33,  18, 154, 173,  86, 122, 192, 147, 118,   9, 121,  75,
        95, 164, 123,  35,  91,  36, 172, 127, 107, 155,  89, 177, 197,
        56,  93, 188,  87,  71, 162,  48, 157, 106, 141, 145,   5,  32,
       163, 131, 167, 186, 119,  49,  24, 151, 130,  90, 111, 135, 180,
       125,  43,  51, 114, 195, 199,  52,  23, 150,  31, 116,  85,  70,
       137, 105, 128, 104,  58, 112, 196, 132,  11,  28, 175,  66,  82,
       194, 156,  83, 142,  45, 144,  88,  74, 178, 138,  77,   8,  97,
       134,  20,  55, 193,  54,  15, 102,  57, 159, 140, 171,  62, 113,
       187,  63, 143,  64,  25, 181,  50, 120, 168,  59,   3,  72,  65,
        68,  84, 101,  19,   4,  37,  61,  69, 189, 152, 182,  42,  16,
       129, 176,  47,  79, 153,  14, 183,  46, 160,  73,  60,  9

Identificando los coeficientes de correlación entre Features y Outputs, se puede identificar qué parámetros del modelo se correlacionan más con la predicción.

# Feature engineering

In [35]:
# Agregamos 200 columnas con el cuadrado de cada columna normaliado a train
X_train_sq = X_train_normalized**2
means_sq = X_train_sq.mean(axis=0)
stds_sq = X_train_sq.std(axis=0)
X_train_FE = np.append(X_train_normalized, (X_train_sq - means_sq)/stds_sq, axis=1)

# Agregamos el cuadrado a validación normalizando con la media y std obtenida en train
X_val_sq = X_val_normalized**2
X_val_FE = np.append(X_val_normalized, (X_val_normalized**2-means_sq)/stds_sq, axis=1)

In [37]:
model_3 = LogisticRegression(solver='lbfgs')
%time model_3.fit(X_train_FE, y_train)

Wall time: 3.52 s


LogisticRegression()

In [ ]:
print(model_3.score(X_val_FE, y_val))
print(roc_auc_score(y_val, model_3.predict_proba(X_val_FE)[:,1]))

0.9221
0.8927208962331821


In [48]:
# Cantidad de parámetros aprendidos por el modelo:
print(len(model_3.coef_[0]) + len(model_3.intercept_))

401


In [218]:
# Coeficientes ordenados en función de su de mayor importancia
np.argsort(np.abs(model_3.coef_))[0][::-1]

array([139,  81,   6,  53,  12,  21,  76, 174, 110,  34, 146,  26, 169,
        99, 165, 281,  22, 166, 190,  78,  33,  40,  13, 170, 149,  80,
       148, 184,  94, 109, 133, 192,  92, 122, 115,  67, 121, 198,   0,
         1, 173, 179,   2, 212,  18, 118, 108, 253, 398, 107, 191, 172,
       177,  44, 154, 310, 164,  35, 197, 226, 188,  36, 339, 299,  75,
       147, 244,   9, 280,  86,  87, 366,  95,  89,  48, 390, 127,  91,
       222, 309,  71, 346, 155, 202,   5, 323, 374, 379,  56, 145, 106,
       162, 123, 206, 221,  32, 186, 141, 200, 150, 348, 151,  31, 365,
       199,  49, 130, 240,  23, 278, 167, 157,  24, 333,  90, 276,  93,
       111, 119, 354, 137, 364, 195, 286, 363, 128, 112, 180, 380,  52,
       104, 114,  58, 131,  70, 355, 116, 125, 135, 295, 370, 233, 391,
       292, 201,  43, 275, 175, 196, 293, 357, 388, 294,  85, 163, 213,
       347,  82, 308, 142, 132,  28, 105,  11, 319, 234, 194, 291,  74,
       205,  97,  77, 140, 315,  45, 322,  20, 369,  55, 267, 25

# Independecia

In [49]:
accs = []
aucs = []
val_predictions = np.zeros((len(X_val_normalized), 200))
train_predictions = np.zeros((len(X_train_normalized), 200))
for i in tqdm.tqdm(range(200)):
    model_ind = LogisticRegression(solver='lbfgs')
    X_one_train = X_train_normalized[:,i].reshape(-1,1)
    # X_one_train = np.array([X_train_normalized[:,i], X_train_normalized[:,i]**2]).T
    model_ind.fit(X_one_train, y_train)
    X_one_val = X_val_normalized[:,i].reshape(-1,1)
    # X_one_val = np.array([X_val_normalized[:,i], X_val_normalized[:,i]**2]).T
    train_predictions[:, i] = model_ind.predict_proba(X_one_train)[:,1]
    val_predictions[:, i] = model_ind.predict_proba(X_one_val)[:,1]
    acc = model_ind.score(X_one_val, y_val)
    auc = roc_auc_score(y_val, val_predictions[:, i])
    accs.append(acc)
    aucs.append(auc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:20<00:00,  9.63it/s]


In [50]:
roc_auc_score(y_train, train_predictions.sum(axis=1)) 

0.8692585548756987

In [51]:
roc_auc_score(y_val, val_predictions.sum(axis=1)) 
# 0.8729094090424484

0.8729094090424484